# <font color='blue'>Data Science Academy - Deep Learning I</font>

# Treinamento de uma Deep Neural Network

Apesar das vantagens computacionais e de ser um modelo mais aproximado do cérebro, o treinamento de redes neurais profundas era uma tarefa de difícil execução até uma década atrás, devido a dois complicadores principais. O primeiro deles é conhecido como a dissipação dos gradientes (vanishing gradients), que é causado por unidades que estão saturadas. Durante a aplicação do algoritmo de retropropagação, o gradiente da ativação de cada unidade oculta é calculado e utilizado como um termo multiplicativo para atualização dos pesos correspondentes. Entretanto se a unidade em questão está próxima da saturação, então a derivada parcial resultante será um valor próximo de zero. Isso faz com que o gradiente com relação às pré-ativações seja também próximo de zero, porque seu cálculo envolve a multiplicação pela derivada parcial. Isso significa que o gradiente retropropagado para as camadas anteriores será cada vez mais próximo de zero, de tal forma que quanto mais próximo à camada de entrada, maior a quantidade de gradientes próximos de zero. Como resultado, à medida que o gradiente do erro é retropropagado seu valor decresce exponencialmente, de tal forma que o aprendizado nas camadas mais próximas à entrada se torna muito lento. Esse problema afeta não apenas o aprendizado em redes profundas alimentadas adiante (Feed Forward), mas também em redes recorrentes (RNNs). O segundo complicador é consequência do número maior de parâmetros para adequar (os parâmetros de uma rede neural são seus pesos, e quanto mais camadas, mais pesos). De fato, quanto maior a quantidade de parâmetros a ajustar em um modelo, maior o risco de overfitting.

O aprendizado de modelos por meio do treinamento de redes neurais profundas tem sido tradicionalmente tratado como um processo empírico. De fato, soluções envolvem o uso de diversos procedimentos que foram surgindo ao longo dos anos e que em conjunto passaram a contar como fatores importantes no sucesso da aprendizagem profunda (além do aumento do poder computacional, da disponibilidade de grandes conjuntos de dados e de melhores arquiteturas de rede). Vamos agora, descrever o procedimentos cujo propósito é treinar redes neurais profundas de forma adequada, seja diminuindo o tempo de treinamento, seja diminuindo o erro de generalização dos modelos resultantes.

## Construindo um Vetor de Features

As redes neurais exigem que suas entradas sejam um número fixo de colunas. Isso é muito semelhante aos dados de uma planilha. Esta entrada deve ser completamente numérica.

É importante representar os dados de forma a que a rede neural possa se formar a partir deles. Vejamos quais são algumas das formas mais básicas de transformar dados para uma rede neural, começando pelos possíveis tipos de dados:

* Dados do Tipo Caracter (strings)
    * **Nominal** - Itens discretos individuais, sem ordem. Por exemplo: cor, código postal, forma.
    * **Ordinal** - Itens discretos individuais que podem ser ordenados. Por exemplo: nível de classificação, título do trabalho, tamanho de uma roupa (pequeno, médio, grande)
    
    
* Dados Numéricos
    * **Interval** - Valores numéricos, início não definido. Por exemplo, a temperatura. Você nunca diria "ontem foi duas vezes mais quente do que hoje".
    * **Ratio** - Valores numéricos, início claramente definido. Por exemplo, a velocidade. Você diria que "O primeiro carro está indo duas vezes mais rápido do que o segundo".

Abaixo você vai encontrar várias funções úteis para codificar o vetor de recursos para vários tipos de dados. 

* **encode_text_dummy** - Encoding de campos de texto, como as espécies de íris como um único campo para cada classe. Três classes se tornariam "0,0,1" "0,1,0" e "1,0,0". Podemos fazer o encoding dos preditores não-alvo dessa maneira. Bom para dados nominais.
* **encode_text_index** - Encoding dos campos de texto, como um único campo numérico como "0" "1" e "2". Bom para dados nominais.
* **encode_numeric_zscore** - Encoding dos valores numéricos com um escore z. As redes neurais lidam bem com os campos "centrados", o zscore geralmente é um bom ponto de partida para o intervalo / proporção.

* Os valores ordinais podem ser codificados com as funções dummy ou index. Mais tarde, veremos um meio mais avançado de encoding *

Tratamento de dados missing:

* **missing_median** - Preenche todos os valores faltantes com o valor médio.

Criando o vetor final de recursos:

* **to_xy** - Uma vez que todos os campos são numéricos, esta função pode fornecer as matrizes x e y que são usadas para caber na rede neural.

Outras funções de utilidade:

* **hms_string** - Imprima uma sequência de tempo decorrido.
* **chart_regression** - Exibe um gráfico para mostrar o quão bem uma regressão executa.

## Normalização

Normalmente, os dados são dimensionados para um intervalo específico em um processo chamado normalização. 

Uma transformação usual durante o treinamento de uma RNA é normalizar o conjunto de dados de treinamento de acordo com a distribuição normal padrão (i.e., média igual a zero e variância igual a 1) para evitar problemas de comparação devido às diferentes escalas usadas nos dados. Ocorre que durante a passagem dos exemplos normalizados através das
camadas da rede, esses valores são novamente transformados (por meio das pré-ativações e ativações), podendo fazer com que os dados de entrada de algumas camadas ocultas fiquem desnormalizados novamente. Esse problema, conhecido como mudança de co-variável interna (internal covariate shift), é tanto mais grave quanto mais profunda for a rede a ser treinada. Esse problema aumenta o tempo de treinamento porque implica na definição de uma taxa de aprendizagem pequena, além de propiciar a dissipação dos gradientes.

A Normalização em Lote (Batch Normalization) é um mecanismo proposto recentemente para resolver o problema acima, e que consiste em normalizar os dados fornecidos a cada camada oculta. A normalização é aplicada em cada mini-lote (como já fizemos em exemplos anteriores), para aumentar a eficiência durante a aplicação da transformação. De acordo com os experimentos realizados pelos autores da técnica (link na seção de links úteis), ela também produz um efeito de regularização sobre o treinamento, em alguns casos eliminando a necessidade de aplicar o desligamento (Dropout). Uma aceleração significativa do tempo de treinamento também foi observada, resultante da diminuição de 14 vezes na quantidade de passos de treinamento necessários, quando comparada ao tempo de treinamento sem o uso da normalização.

Existem muitas maneiras diferentes de normalizar os dados. Vejamos algumas delas:

### One Hot Encoding

Se você tem um valor categórico, como a espécie de uma íris, a marca de um automóvel ou a etiqueta de dígitos no conjunto de dados MNIST, você deve usar uma codificação One-Hot Encoding. Para codificar dessa maneira, você usaria um neurônio de saída para cada classe no problema. Lembra do conjunto de dados MNSIT que já usamos em outras oportunidades, onde você tem imagens para dígitos entre 0 e 9? Este problema é mais comumente codificado como dez neurônios de saída com uma função de ativação softmax que dá a probabilidade de a entrada ser um desses dígitos. Usando a codificação One-Hot, os dez dígitos podem ser codificados da seguinte maneira:

![One-Hot Encoding](images/one-hot.png "One-Hot Encoding")

One-Hot Encoding sempre deve ser usado quando as classes não têm ordenação. Outro exemplo desse tipo de codificação é a marca de um automóvel. Normalmente, a lista de fabricantes de automóveis não está ordenada, a menos que exista algum significado que você deseja transmitir. Por exemplo, você pode encomendar as montadoras pelo número de anos de negócios. No entanto, esta classificação só deve ser feita se o número de anos de negócios tiver significado no seu problema. Se realmente não há ordem, então One-Hot deve sempre ser usado. Como você pode facilmente converter os dígitos, você pode se perguntar por que usamos uma codificação One-Hot. No entanto, a ordem dos dígitos não significa que o programa possa reconhecê-los. O fato de que "1" e "2" estarem numericamente próximos um do outro não ajuda o programa a reconhecer a imagem. Portanto, não devemos usar um neurônio de saída único que simplesmente exiba o dígito reconhecido. Os dígitos 0-9 são categorias, e não valores numéricos reais. 

As categorias de codificação com um único valor numérico prejudicam o processo de decisões da rede neural. Tanto a entrada como a saída podem usar One-Hot Encoding. A lista acima usou 0 's e 1' s. Normalmente você usará a unidade linear retificada (ReLU) e a ativação do softmax. No entanto, se você estiver trabalhando com uma função de ativação hiperbólica tangente, você deve utilizar um valor de -1 para os 0 para coincidir com o intervalo da hiperbólica tangente de -1 a 1.

Se você tem um número extremamente grande de classes, One-Hot Encoding pode tornar-se complicado porque você deve ter um neurônio para cada classe. Nesses casos, você tem várias opções. Primeiro, você pode encontrar uma maneira de pedir suas categorias. Com esta ordem, suas categorias agora podem ser codificadas como um valor numérico, que seria a posição da categoria atual dentro da lista ordenada. Outra abordagem para lidar com um número extremamente grande de categorias é a codificação de frequência de documento inversa (TF-IDF) porque cada classe se torna essencialmente a probabilidade de ocorrência dessa classe em relação às demais. Desta forma, o TF-IDF permite que o programa mapeie um grande número de classes para um único neurônio. 

## Range

Se você tem um número real ou uma lista ordenada de categorias, você pode escolher a normalização de range, pois simplesmente mapeia o intervalo de dados de entrada no alcance de sua função de ativação. Sigmoid, ReLU e softmax usam um intervalo entre 0 e 1, enquanto a tangente hiperbólica usa um intervalo entre -1 e 1.

Para executar a normalização, é necessário que os valores altos e baixos dos dados sejam normalizados. Da mesma forma, você precisa de valores altos e baixos para normalizar (geralmente 0 e 1). Às vezes você precisará desfazer a normalização realizada em um número e retorná-lo para um estado desnormalizado.

Uma maneira muito simples de pensar sobre a normalização de range é porcentagem. Considere a seguinte analogia. Você vê um anúncio afirmando que você receberá uma redução de 10 reais em um produto, e você deve decidir se esse negócio vale a pena. Se você está comprando uma camiseta, esta oferta provavelmente é um bom negócio. No entanto, se você está comprando um carro, 10 reais realmente não importam. A situação muda se você descobrir que o comerciante ofereceu um desconto de 10%. Assim, o valor agora é mais significativo. Não importa se você está comprando uma camiseta, um carro ou mesmo uma casa, o desconto de 10% tem ramificações claras sobre o problema porque transcende as moedas. Em outras palavras, a porcentagem é um tipo de normalização. Assim como na analogia, a normalização para um intervalo ajuda a rede neural a avaliar todas as entradas com igual significado.

## Escore Z

A normalização de escore Z é a normalização mais comum para um número real ou uma lista ordenada. Para quase todas as aplicações, a normalização do z-score deve ser usada no lugar da normalização de range. Esse tipo de normalização baseia-se no conceito estatístico de escores z. Os escores Z fornecem ainda mais informações do que percentagens. 

Considere o seguinte exemplo: 

O estudante A obteve 85% dos pontos em seu exame. O estudante B obteve 75% dos pontos em seu exame. Qual aluno teve melhor performance? Se o professor está apenas relatando a porcentagem de pontos corretos, o aluno A ganhou uma melhor pontuação. No entanto, você pode mudar sua resposta se você soubesse que a pontuação média (média) para o exame do aluno A era de 95%. Da mesma forma, você pode reconsiderar sua posição se descobrisse que a classe do aluno B tinha uma pontuação média de 65%. O estudante B ficou acima da média no exame. Embora o aluno A tenha obtido uma pontuação melhor, ela ficou abaixo da média. Para reportar verdadeiramente uma pontuação curva (um escore z), você deve ter o escore médio e o desvio padrão. 

Calculamos o z-score, subtraindo a média do valor de x e dividindo pelo desvio padrão. O z-score é um valor numérico onde 0 representa uma pontuação que é exatamente a média. Um escore z positivo está acima da média. Um escore z negativo está abaixo da média. 

![Z-Score](images/z-score.png "Z-Score")

## Funções Auxiliares

É uma boa prática criar suas próprias funções de limpeza e transformação dos dados que serão processados pelo modelo de rede neural. Use esses exemplos como referência.

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

In [2]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
# Imports
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
import sklearn
import matplotlib

Using TensorFlow backend.


In [4]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

pandas     1.0.3
tensorflow 2.1.0
matplotlib 3.2.1
sklearn    0.22.2
keras      2.3.1
numpy      1.18.2
Data Science Academy


In [5]:
# Imports
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os

# Encoding dos valores de texto para variáveis nominais
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encoding dos valores de texto para uma única variável dummy. As novas colunas (que não substituem o antigo) terão 1
# em todos os locais onde a coluna original (nome) corresponde a cada um dos valores-alvo. Uma coluna é adicionada para
# cada valor alvo.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encoding dos valores de texto para índices (ou seja, [1], [2], [3] para vermelho, verde, azul por exemplo).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Normalização Z-score
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Converte todos os valores faltantes na coluna especificada para a mediana
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Converte todos os valores faltantes na coluna especificada para o padrão
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Converte um dataframe Pandas para as entradas x, y que o TensorFlow precisa
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    
    # Descobre o tipo da coluna de destino. 
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    
    # Encoding para int. TensorFlow gosta de 32 bits.
    if target_type in (np.int64, np.int32):
        # Classificação
        dummies = pd.get_dummies(df[target])
        return df.values.astype('float32'), dummies.values.astype('float32')
    else:
        # Regressão
        return df.values.astype('float32'), df[target].astype('float32')

# String de tempo bem formatado
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Chart de Regressão
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace = True)
    a = plt.plot(t['y'].tolist(),label = 'expected')
    b = plt.plot(t['pred'].tolist(),label = 'prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove todas as linhas onde a coluna especificada em +/- desvios padrão
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace = True)


# Normalização Range
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1, data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) * (normalized_high - normalized_low) + normalized_low

# Treinamento com um Conjunto de Validação e Early Stopping

O **Overfitting** ocorre quando uma rede neural é treinada no ponto em que começa a memorizar em vez de generalizar.

![Treinamento x Erro de Validação](images/training_val.png "Treinamento x Erro de Validação")

É importante segmentar o conjunto de dados original em vários conjuntos de dados:

* **Treinamento**
* **Validação**
* **Teste**

Existem várias maneiras diferentes de que esses conjuntos possam ser construídos. Os seguintes programas demonstram alguns deles.

O primeiro método é um conjunto de treinamento e validação. Os dados de treinamento são usados para treinar a rede neural até que o conjunto de validação não melhore mais. Isso tenta parar em um ponto de treinamento quase ótimo. Este método apenas dará previsões precisas de "fora de amostra" para o conjunto de validação, isto geralmente é 20% ou mais dos dados. As previsões para os dados de treinamento serão excessivamente otimistas, pois esses foram os dados em que a rede neural foi treinada.

![Treinamento com um conjundo de validação](images/train_val.png "Treinamento com um conjundo de validação")


Early Stopping no Keras: https://keras.io/callbacks/

In [6]:
# Imports
import io
import os
import requests
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping

# Path
path = "./data/"
    
# Arquivo
filename = os.path.join(path,"iris.csv")   

# Leitura do arquivo em um dataframe
df = pd.read_csv(filename,na_values=['NA','?'])

# Encoding dos valores de texto para índices
species = encode_text_index(df, "species")

# Conversão do input
x,y = to_xy(df,"species")

# Divisão em dados de treino e de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

# Construindo a Rede Neural
model = Sequential()
model.add(Dense(10, input_dim = x.shape[1], kernel_initializer = 'normal', activation = 'relu'))
model.add(Dense(1, kernel_initializer = 'normal'))
model.add(Dense(y.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

# Early stoppping
monitor = EarlyStopping(monitor = 'val_loss', min_delta = 1e-3, patience = 5, verbose = 1, mode = 'auto')

# Fit do modelo
model.fit(x, y, validation_data = (x_test, y_test), callbacks = [monitor], verbose = 2, epochs = 1000)

Train on 150 samples, validate on 38 samples
Epoch 1/1000
 - 0s - loss: 1.0957 - val_loss: 1.0934
Epoch 2/1000
 - 0s - loss: 1.0937 - val_loss: 1.0908
Epoch 3/1000
 - 0s - loss: 1.0907 - val_loss: 1.0890
Epoch 4/1000
 - 0s - loss: 1.0889 - val_loss: 1.0868
Epoch 5/1000
 - 0s - loss: 1.0867 - val_loss: 1.0846
Epoch 6/1000
 - 0s - loss: 1.0850 - val_loss: 1.0821
Epoch 7/1000
 - 0s - loss: 1.0823 - val_loss: 1.0797
Epoch 8/1000
 - 0s - loss: 1.0799 - val_loss: 1.0771
Epoch 9/1000
 - 0s - loss: 1.0774 - val_loss: 1.0741
Epoch 10/1000
 - 0s - loss: 1.0742 - val_loss: 1.0709
Epoch 11/1000
 - 0s - loss: 1.0715 - val_loss: 1.0673
Epoch 12/1000
 - 0s - loss: 1.0675 - val_loss: 1.0636
Epoch 13/1000
 - 0s - loss: 1.0639 - val_loss: 1.0593
Epoch 14/1000
 - 0s - loss: 1.0594 - val_loss: 1.0545
Epoch 15/1000
 - 0s - loss: 1.0550 - val_loss: 1.0495
Epoch 16/1000
 - 0s - loss: 1.0496 - val_loss: 1.0442
Epoch 17/1000
 - 0s - loss: 1.0445 - val_loss: 1.0385
Epoch 18/1000
 - 0s - loss: 1.0392 - val_loss:

## Avaliando o Resultado da Classificação em Redes Neurais

### Calculando a Acurácia de Classificação
 
A Acurácia é o número de linhas onde a rede neural previu corretamente a classe alvo. A Acurácia é usada apenas para classificação, não regressão.

$ accuracy = \frac{\textit{#} \ correct}{N} $

Onde $N$ é o tamanho do conjunto avaliado (treinamento ou validação). A Acurácia deve ser tão alta quanto possível.

In [7]:
from sklearn import metrics
import tensorflow as tf

# Avaliando o resultado usando a Acurácia
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state = 22)
pred = model.predict(x_test)
pred = np.argmax(pred,axis = 1) 
y_compare = np.argmax(y_test, axis = 1) 
score = metrics.accuracy_score(y_compare, pred)
print("Accurácia: {}".format(score))

Accurácia: 1.0


### Calculando o Log Loss de Classificação

As redes neurais podem prever uma probabilidade de cada uma das classes alvo. As redes neurais darão probabilidades elevadas a previsões que são mais prováveis. O Log Loss é uma métrica de erro que penaliza a confiança em respostas erradas. Valores baixos de Log Loss são desejados.

Para qualquer modelo scikit-learn, existem duas maneiras de obter uma previsão:

* **predict** - No caso da saída de classificação, o id numérico da classe prevista. Para a regressão, isso é simplesmente a previsão.
* **predict_proba** - No caso da classificação, a probabilidade de cada uma das classes. Não utilizado para regressão.

O código a seguir mostra a saída de predict_proba:

In [8]:
from IPython.display import display

# Não exibe numpy na notação científica
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

# Gerar previsões
pred = model.predict(x_test)

print("Array Numpy de Previsões")
print(pred[0]*100)

print("Como Percentual de Probabilidade")
display(pred[0:5])

score = metrics.log_loss(y_test, pred)
print("Log loss score: {}".format(score))

Array Numpy de Previsões
[99.5038  0.4962  0.    ]
Como Percentual de Probabilidade


array([[0.995 , 0.005 , 0.    ],
       [0.    , 0.0018, 0.9982],
       [0.0855, 0.8729, 0.0416],
       [0.    , 0.0017, 0.9983],
       [0.0827, 0.8726, 0.0447]], dtype=float32)

Log loss score: 0.05776506934263226


Log loss é calculado assim:

$ \text{log loss} = -\frac{1}{N}\sum_{i=1}^N {( {y}_i\log(\hat{y}_i) + (1 - {y}_i)\log(1 - \hat{y}_i))} $

## Avaliando o Resultado da Regressão em Redes Neurais

# Avaliando Resultado de Regressão

Os resultados da regressão são avaliados de forma diferente da classificação. Considere o seguinte código que treina uma rede neural para o conjunto de dados MPG.

In [9]:
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

# Path
path = "./data/"

# Arquivo
filename_read = os.path.join(path,"auto-mpg.csv")

# Dataframe
df = pd.read_csv(filename_read,na_values=['NA','?'])

# Limpeza e transformação
cars = df['name']
df.drop('name',1,inplace=True)
missing_median(df, 'horsepower')
x,y = to_xy(df,"mpg")

# Split em treino/teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 45)

# Rede Neural
model = Sequential()
model.add(Dense(10, input_dim = x.shape[1], kernel_initializer = 'normal', activation = 'relu'))
model.add(Dense(1, kernel_initializer = 'normal'))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
monitor = EarlyStopping(monitor = 'val_loss', min_delta = 1e-3, patience = 5, verbose = 1, mode = 'auto')
model.fit(x, y, validation_data = (x_test, y_test), callbacks = [monitor], verbose = 2, epochs = 1000)

Train on 398 samples, validate on 100 samples
Epoch 1/1000
 - 0s - loss: 309.3967 - val_loss: 198.5292
Epoch 2/1000
 - 0s - loss: 198.0366 - val_loss: 143.8272
Epoch 3/1000
 - 0s - loss: 179.7232 - val_loss: 145.8026
Epoch 4/1000
 - 0s - loss: 176.9136 - val_loss: 140.6661
Epoch 5/1000
 - 0s - loss: 172.3939 - val_loss: 138.8396
Epoch 6/1000
 - 0s - loss: 170.7753 - val_loss: 136.9922
Epoch 7/1000
 - 0s - loss: 167.6785 - val_loss: 135.3988
Epoch 8/1000
 - 0s - loss: 166.6302 - val_loss: 133.5028
Epoch 9/1000
 - 0s - loss: 163.0679 - val_loss: 131.3284
Epoch 10/1000
 - 0s - loss: 161.6036 - val_loss: 128.6121
Epoch 11/1000
 - 0s - loss: 157.8485 - val_loss: 127.0200
Epoch 12/1000
 - 0s - loss: 155.9164 - val_loss: 123.7614
Epoch 13/1000
 - 0s - loss: 154.4763 - val_loss: 122.3495
Epoch 14/1000
 - 0s - loss: 148.7897 - val_loss: 118.9726
Epoch 15/1000
 - 0s - loss: 146.6918 - val_loss: 115.3787
Epoch 16/1000
 - 0s - loss: 142.6043 - val_loss: 112.4986
Epoch 17/1000
 - 0s - loss: 137.951

### Mean Square Error

O erro quadrático médio é a soma das diferenças quadradas entre a previsão ($\hat{y}$) e o esperado ($y$). Os valores de MSE não são de uma unidade específica. Se um valor MSE diminuiu para um modelo, isso é bom. No entanto, além disso, não há muito mais que você possa determinar. São desejados valores de MSE baixos.

$ \text{MSE} = \frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2 $


In [10]:
# Previsões
pred = model.predict(x_test)

# Medindo o MSE
score = metrics.mean_squared_error(pred,y_test)
print("Final score (MSE): {}".format(score))

Final score (MSE): 2.0803844928741455


### Root Mean Square Error

O quadrado médio da raiz (RMSE) é essencialmente a raiz quadrada do MSE. Por causa disso, o erro RMSE está nas mesmas unidades que o resultado dos dados de treinamento. São desejados valores RMSE baixos.

$ \text{MSE} = \sqrt{\frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2} $

In [11]:
# Medindo o RMSE
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 1.4423538446426392


# Treinamento com Cross Validation

A validação cruzada usa várias "dobras" (folds) e vários modelos, para gerar previsões de amostra em todo o conjunto de dados. É importante notar que haverá um modelo (rede neural) para cada dobra. Cada modelo contribui parte da previsão final fora da amostra.

![K-Fold Crossvalidation](images/kfold.png "K-Fold Crossvalidation")

Para novos dados, que são dados não presentes no conjunto de treinamento, as previsões dos modelos dos folds podem ser tratadas de várias maneiras.

* Escolha o modelo que teve o maior resultado de validação como modelo final.
* Predefinição de novos dados para os 5 modelos e média do resultado 
* Retome um novo modelo (usando as mesmas configurações que a validação cruzada) em todo o conjunto de dados. Treine para vários passos e com a mesma estrutura de camada oculta.

O código a seguir treina o conjunto de dados MPG usando uma validação cruzada 5 vezes. O desempenho esperado de uma rede neural, do tipo treinado aqui, seria a pontuação para as previsões geradas fora da amostra.

In [12]:
# Imports
import os
import pandas as pd
import numpy as np
from sklearn import metrics
from scipy.stats import zscore
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Activation

# Path
path = "./data/"

# Arquivos
filename_read = os.path.join(path,"auto-mpg.csv")
filename_write = os.path.join(path,"auto-mpg-saida.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

# Shuffle
np.random.seed(42)
df = df.reindex(np.random.permutation(df.index))
df.reset_index(inplace=True, drop=True)

# Preprocesso
cars = df['name']
df.drop('name',1,inplace=True)
missing_median(df, 'horsepower')

# Encode para uma matriz 2D 
x,y = to_xy(df,'mpg')

# Cross validate
kf = KFold(5)
    
oos_y = []
oos_pred = []
fold = 0

# Treino
for train, test in kf.split(x):
    fold += 1
    print("Fold #{}".format(fold))
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(20, input_dim = x.shape[1], activation = 'relu'))
    model.add(Dense(1, kernel_initializer = 'normal'))
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    
    monitor = EarlyStopping(monitor = 'val_loss', min_delta = 1e-3, patience = 5, verbose = 1, mode = 'auto')
    model.fit(x_train, y_train,validation_data = (x_test,y_test), callbacks = [monitor], verbose = 0, epochs = 1000)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    oos_pred.append(pred)        

    # Medindo o RMSE por Fold
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print("Fold score (RMSE): {}".format(score))


# Cria a lista de previsões e calcula o erro.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print("Final Score (RMSE): {}".format(score))    
    
# Grava a previsão
oos_y = pd.DataFrame(oos_y)
oos_pred = pd.DataFrame(oos_pred)
oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )
oosDF.to_csv(filename_write, index=False)

Fold #1
Epoch 00147: early stopping
Fold score (RMSE): 2.8485829830169678
Fold #2
Epoch 00252: early stopping
Fold score (RMSE): 1.9778051376342773
Fold #3
Epoch 00103: early stopping
Fold score (RMSE): 3.9483745098114014
Fold #4
Epoch 00145: early stopping
Fold score (RMSE): 3.4486491680145264
Fold #5
Epoch 00217: early stopping
Fold score (RMSE): 3.5744497776031494
Final Score (RMSE): 3.232290029525757


# Fim